In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import logging
import constants
from pathlib import Path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [28]:
dir(exceptions)

['ClientCertError',
 'DefaultCredentialsError',
 'GoogleAuthError',
 'MutualTLSChannelError',
 'OAuthError',
 'ReauthFailError',
 'ReauthSamlChallengeFailError',
 'RefreshError',
 'TransportError',
 'UserAccessTokenError',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__']

In [ ]:
def do_exit(msg=None, code=0):
    '''provide exit message and exit
    
    Args:
    '''
    status = ''
    if code > 0:
        status = f'Error {code}: '
    if msg:
        print(f'{status}{msg}')
    exit(code)

In [ ]:
logging.root.setLevel('INFO')
cache = Path(constants.CACHE).expanduser()
token_file = cache/constants.TOKEN_FILE

creds = None


# create cache directory if missing:
if not cache.exists():
    try:
        cache.mkdir(parents=True, exist_ok=True)
    except OSError as e:
        msg = f'error creating cache directory: {type(e)} {e}'
        logging.error(msg)
        do_exit(msg, 1)

if token_file.exists():
    creds = Credentials.from_authorized_user_file('token.json', constants.SCOPES)
else:
    flow = InstalledAppFlow.from_client_secrets_file(
        './secrets/client_secret_910311278281-bh8qk3kmgk0veri3v8en260e76ipafpj.apps.googleusercontent.com.json', constants.SCOPES)
    creds = flow.run_local_server(port=0)
# Save the credentials for the next run
with open('token.json', 'w') as token:
    token.write(creds.to_json())
    
try:
    service = build('drive', 'v3', credentials=creds)
except HttpError as e:
    msg = f'error in creating Google Drive interface: HttpError - {e}'
    logging.error(e)
    do_exit(e, 1)



In [52]:
from pathlib import Path
from json import JSONDecodeError
import logging

import constants_GDrive 
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.auth import exceptions as google_exceptions


In [10]:
logging.basicConfig(level=logging.DEBUG)

logger = logging.getLogger(__name__)
logger.setLevel('DEBUG')
logger.debug('foo')

DEBUG:__main__:foo


In [34]:
class GDriveError(Exception):
    pass

In [259]:
class GDrive():
    def __repr__(self):
        return 'GDrive()'
    
    def __str__(self):
        return f'GDrive()'
    
    def __init__(self, secrets, scopes, cache='./', token='./token.json'):
        '''create a google drive interface for searching and returning file/folder information
        
        Args:
            secrets(Path): secrets json file obtained from https://console.cloud.google.com/cloud-resource-manager
            token(Path): file to cache auth information (typically within cache path)
        '''
        self.secrets = secrets
        self.scopes = scopes        
        self.token = token
        self.credentials = self.create_token(secrets=self.secrets, 
                                            scopes=self.scopes, 
                                            token=self.token)
        self.service = self.build_service(self.credentials)
        self.MIMETYPES = constants_GDrive.MIMETYPES
        self.CORPORA = constants_GDrive.CORPORA
        self.FILE_FIELDS = constants_GDrive.FILE_FIELDS
        self.FIELDS_DEFAULT = constants_GDrive.FIELDS_DEFAULT
        
                
    
    
    @staticmethod
    def create_token(secrets, scopes, token):
        '''create credentials object
        
        Args:
            secrets(Path): path to secrets file
            scopes(list): list of scopes to use
            token(Path): path to store cached auth token
            
        Returns: credentials object
        '''
        token = Path(token)
        write_token = False
        if token.exists():
            try:
                logging.debug(f'creating credentials token: {token} with scopes: {scopes}')
                creds = Credentials.from_authorized_user_file(token, scopes)
            except JSONDecodeError as e:
                logger.warning(f'rewriting token file due to error: {e}')
                write_token = True
        else: 
            write_token = True
        
        if write_token:
            flow = InstalledAppFlow.from_client_secrets_file(secrets, scopes)
            creds = flow.run_local_server(port=0)
    
            try:
                logger.debug(f'writing token file: {token}')
                with open(token, 'w') as tf:
                    tf.write(creds.to_json())
            except OSError as e:
                raise GDriveError(f'error writing token file ({token}): {e}')
        
        return creds
    
    @staticmethod
    def build_service(credentials):
        try:
            service  = build('drive', 'v3', credentials=credentials)
        except google_exceptions.GoogleAuthError as e:
            raise GDriveError(f'error building credentials: {e}')
        return service
    
                
    @property
    def token(self):
        '''token file'''
        return self._token
        
    @token.setter
    def token(self, t_path):
        t_path = Path(t_path)        
        self._token = t_path

 

In [260]:
sec = './secrets/client_secret_910311278281-bh8qk3kmgk0veri3v8en260e76ipafpj.apps.googleusercontent.com.json'
d = GDrive(secrets=sec, scopes=constants.SCOPES)

DEBUG:root:creating credentials token: token.json with scopes: ['https://www.googleapis.com/auth/drive']
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


In [261]:
class DC():
    pass
self = DC()
self.mimetypes = constants_GDrive.MIMETYPES
self = d

In [267]:
d.CORPORA

{'user': {'params': {'includeItemsFromAllDrives': '', 'supportsAllDrives': ''},
  'description': 'files created by, opened by, or shared directly with the user'},
 'drive': {'params': {'includeItemsFromAllDrives': True,
   'supportsAllDrives': True},
  'description': "files in the specified shared drive as indicated by the 'driveId'"},
 'domain': {'params': {'includeItemsFromAllDrives': True,
   'supportsAllDrives': True},
  'description': "files shared to the user's domain"},
 'allDrives': {'params': {'includeItemsFromAllDrives': True,
   'supportsAllDrives': True},
  'description': "A combination of 'user' and 'drive' for all drives where the user is a member"}}

In [269]:
def search(self, name=None, trashed=False, mimeType=None, fuzzy=True, 
           modifiedTime=None, parents=None, dopperator='>',
           fields = [], forcefields=False,
           corpora='user', orderBy='createdTime', driveId=''):
    '''search for objects in google drive by name
    
    Args:
        name(str): string to search for
        trashed(bool): search in trash when true
        mimeType(str): short mimeType (see MIMETYPES property)
        fuzzy(bool): true: `name contains "value"` false: `name = "value"`
        modifiedTime(str): yyyy-mm-dd string
        dopperator(str): >, < for use with modifiedTime
        parents(str): folder to search within
        fields(list of str): fields to return (see FILE_FIELDS property)
        forcefields(bool): true: use unknown fields, false: reject fields not in FILE_FIELDS
        corpora(str): locations to search (see CORPORA property)
        orderBy(str): order results by (see https://developers.google.com/drive/api/v3/reference/files/list)
        driveId(str): search this shared drive
        
    Retruns dict of resutls
        '''

    query_build = {
        'name': (name, f'name {"contains" if fuzzy else "="} "{name}"'),
        'trashed': (trashed, f'trashed={trashed}'),
        'mimeType': (mimeType, f'mimeType="{self.MIMETYPES[mimeType] if mimeType in self.MIMETYPES else ""}"'),
        'parents': (parents, f'"{parents}" in parents'),
        'modifiedTime': (modifiedTime, f'modifiedTime{dopperator}"{modifiedTime}"')
    }
    
    query = [v[1] for k, v in query_build.items() if v[0]]
    
    if len(fields) < 1:
        fields = self.FIELDS_DEFAULT
    fields = set(fields)
    
    known_fields = []
    for f in fields:
        if f not in self.FILE_FIELDS:
            if forcefields:
                logging.warning(f'unknown return field: {f}')
                known_fields.append(f)
            else:
                raise GDriveError(f'unknown return field: {f}')
        else:
            known_fields.append(f)
            
    fields_string = f'files({",".join(known_fields)})'
    
    
    if driveId:
        corpora = 'drive'
    
    if corpora not in self.CORPORA:
        raise GDriveError(f'unknown `corpora` value: {corpora}')
    else:
        includeItemsFromAllDrives = self.CORPORA[corpora]['params']['includeItemsFromAllDrives']
        supportsAllDrives = self.CORPORA[corpora]['params']['supportsAllDrives']
    

    q = ' and '.join(query)
    print(q)
    results = self.service.files().list(q=q,
                                        corpora=corpora,
                                        includeItemsFromAllDrives=includeItemsFromAllDrives,
                                        supportsAllDrives=supportsAllDrives,
                                        fields=fields_string,
                                        driveId=driveId).execute()
    
    return results
    
r = search(self, name='subject access request', corpora='allDrives', driveId='0AOI_iHJTufKOUk9PVA')
r

DEBUG:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/drive/v3/files?q=name+contains+%22subject+access+request%22&corpora=drive&includeItemsFromAllDrives=true&supportsAllDrives=true&fields=files%28name%2Cid%2Ckind%2CmimeType%29&driveId=0AOI_iHJTufKOUk9PVA&alt=json


name contains "subject access request"


{'files': [{'kind': 'drive#file',
   'id': '1WZKhnJS4IXU_lTaL_LIRXh39DNBHA6CdOilAgFhMK08',
   'name': 'Subject Access Request',
   'mimeType': 'application/vnd.google-apps.form'}]}

In [233]:
r

{}

In [197]:
constants_GDrive.CORPORA['user']

{'params': {'includeItemsFromAllDrives': 'false',
  'supportsAllDrives': 'false'},
 'description': 'files created by, opened by, or shared directly with the user'}

In [194]:
r

('name contains "subject access" and mimeType="application/vnd.google-apps.form"',
 'files(id)')

In [211]:
d.service.files().list(q='name contains "subject access request"', corpora="allDrives", supportsAllDrives=True, includeItemsFromAllDrives=True).execute()

DEBUG:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/drive/v3/files?q=name+contains+%22subject+access+request%22&corpora=allDrives&supportsAllDrives=true&includeItemsFromAllDrives=true&alt=json


{'kind': 'drive#fileList',
 'incompleteSearch': False,
 'files': [{'kind': 'drive#file',
   'id': '1WZKhnJS4IXU_lTaL_LIRXh39DNBHA6CdOilAgFhMK08',
   'name': 'Subject Access Request',
   'mimeType': 'application/vnd.google-apps.form',
   'teamDriveId': '0AOI_iHJTufKOUk9PVA',
   'driveId': '0AOI_iHJTufKOUk9PVA'},
  {'kind': 'drive#file',
   'id': '1PosRceG780Pkcx7v9kJofQzdy88IKLkqac6-zhkCjP8',
   'name': 'GDPR Nightmare Letter - A subject Access Request',
   'mimeType': 'application/vnd.google-apps.document'}]}

In [124]:
# r = d.service.files().list(q='name contains "home" and "0B9WTleJ1MzaYT2pieWNXYkZtZm8" in parents').execute()
r = d.service.files().list(q=q).execute()
r

DEBUG:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/drive/v3/files?q=mimeType%3D%22application%2Fvnd.google-apps.folder%22+and+%220B9WTleJ1MzaYT2pieWNXYkZtZm8%22+in+parents&alt=json


{'kind': 'drive#fileList',
 'incompleteSearch': False,
 'files': [{'kind': 'drive#file',
   'id': '1U6NuGcEvrQSdTycR5sZGlaGOzZatPauH',
   'name': 'Never look at this again',
   'mimeType': 'application/vnd.google-apps.folder'},
  {'kind': 'drive#file',
   'id': '1Rf8crb_2qNV_zWG3OH3RZcsZlBs9wkDO',
   'name': '2021.06 and Earlier',
   'mimeType': 'application/vnd.google-apps.folder'},
  {'kind': 'drive#file',
   'id': '18Zj_J0beavJmIDXjzkzVVFYNWHdP8_HZ',
   'name': 'Child Protection Materials',
   'mimeType': 'application/vnd.google-apps.folder'},
  {'kind': 'drive#file',
   'id': '1EUyAB4OhRQvFSk2LrKAQfxjFZFuVgAQr',
   'name': 'appsheet',
   'mimeType': 'application/vnd.google-apps.folder'}]}

In [123]:
r

{'kind': 'drive#fileList',
 'incompleteSearch': False,
 'files': [{'kind': 'drive#file',
   'id': '1U6NuGcEvrQSdTycR5sZGlaGOzZatPauH',
   'name': 'Never look at this again',
   'mimeType': 'application/vnd.google-apps.folder'},
  {'kind': 'drive#file',
   'id': '1Rf8crb_2qNV_zWG3OH3RZcsZlBs9wkDO',
   'name': '2021.06 and Earlier',
   'mimeType': 'application/vnd.google-apps.folder'},
  {'kind': 'drive#file',
   'id': '18Zj_J0beavJmIDXjzkzVVFYNWHdP8_HZ',
   'name': 'Child Protection Materials',
   'mimeType': 'application/vnd.google-apps.folder'},
  {'kind': 'drive#file',
   'id': '1EUyAB4OhRQvFSk2LrKAQfxjFZFuVgAQr',
   'name': 'appsheet',
   'mimeType': 'application/vnd.google-apps.folder'}]}

In [132]:
l = "'kind',\n'id',\n'name',\n'mimeType',\n'description',\n'starred',\n'trashed',\n'explicitlyTrashed',\n'parents',\n'properties',\n'properties',\n'appProperties',\n'appProperties',\n'spaces',\n'version',\n'webContentLink',\n'webViewLink',\n'iconLink',\n'thumbnailLink',\n'viewedByMe',\n'viewedByMeTime',\n'createdTime',\n'modifiedTime',\n'modifiedByMeTime',\n'sharedWithMeTime',\n'sharingUser',\n'sharingUser',\n'sharingUser',\n'sharingUser',\n'sharingUser',\n'sharingUser',\n'sharingUser',\n'owners',\n'owners',\n'owners',\n'owners',\n'owners',\n'owners',\n'owners',\n'lastModifyingUser',\n'lastModifyingUser',\n'lastModifyingUser',\n'lastModifyingUser',\n'lastModifyingUser',\n'lastModifyingUser',\n'lastModifyingUser',\n'shared',\n'ownedByMe',\n'viewersCanCopyContent',\n'writersCanShare',\n'permissions',\n'folderColorRgb',\n'originalFilename',\n'fullFileExtension',\n'fileExtension',\n'md5Checksum',\n'size',\n'quotaBytesUsed',\n'headRevisionId',\n'contentHints',\n'contentHints',\n'contentHints',\n'contentHints',\n'contentHints',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'imageMediaMetadata',\n'videoMediaMetadata',\n'videoMediaMetadata',\n'videoMediaMetadata',\n'videoMediaMetadata',\n'capabilities',\n'capabilities',\n'capabilities',\n'capabilities',\n'capabilities',\n'capabilities',\n'isAppAuthorized',\n'hasThumbnail',\n'thumbnailVersion',\n'modifiedByMe',\n'trashingUser',\n'trashingUser',\n'trashingUser',\n'trashingUser',\n'trashingUser',\n'trashingUser',\n'trashingUser',\n'trashedTime',\n'teamDriveId',\n'capabilities',\n'capabilities',\n'capabilities',\n'capabilities',\n'capabilities',\n'capabilities',\n'capabilities',\n'capabilities',\n'capabilities',\n'capabilities',\n'capabilities',\n'hasAugmentedPermissions',\n'capabilities',\n'permissionIds',\n'capabilities',\n'copyRequiresWriterPermission',\n'capabilities',\n'capabilities',\n'capabilities',\n'capabilities',\n'capabilities',\n'capabilities',\n'exportLinks',\n'exportLinks',\n'driveId',\n'capabilities',\n'capabilities',\n'capabilities',\n'capabilities',\n'capabilities',\n'capabilities',\n'capabilities',\n'capabilities',\n'shortcutDetails',\n'shortcutDetails',\n'shortcutDetails',\n'capabilities',\n'capabilities',\n'contentRestrictions',\n'contentRestrictions',\n'contentRestrictions',\n'contentRestrictions',\n'contentRestrictions',\n'contentRestrictions',\n'contentRestrictions',\n'contentRestrictions',\n'contentRestrictions',\n'contentRestrictions',\n'contentRestrictions',\n'contentRestrictions',\n'capabilities',\n'shortcutDetails',\n'resourceKey',\n'linkShareMetadata',\n'linkShareMetadata',\n'linkShareMetadata',\n'capabilities',\n'Methods',\n'copyCreates',\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n,\n"

print(l)

'kind',
'id',
'name',
'mimeType',
'description',
'starred',
'trashed',
'explicitlyTrashed',
'parents',
'properties',
'properties',
'appProperties',
'appProperties',
'spaces',
'version',
'webContentLink',
'webViewLink',
'iconLink',
'thumbnailLink',
'viewedByMe',
'viewedByMeTime',
'createdTime',
'modifiedTime',
'modifiedByMeTime',
'sharedWithMeTime',
'sharingUser',
'sharingUser',
'sharingUser',
'sharingUser',
'sharingUser',
'sharingUser',
'sharingUser',
'owners',
'owners',
'owners',
'owners',
'owners',
'owners',
'owners',
'lastModifyingUser',
'lastModifyingUser',
'lastModifyingUser',
'lastModifyingUser',
'lastModifyingUser',
'lastModifyingUser',
'lastModifyingUser',
'shared',
'ownedByMe',
'viewersCanCopyContent',
'writersCanShare',
'permissions',
'folderColorRgb',
'originalFilename',
'fullFileExtension',
'fileExtension',
'md5Checksum',
'size',
'quotaBytesUsed',
'headRevisionId',
'contentHints',
'contentHints',
'contentHints',
'contentHints',
'contentHints',
'imageMediaMetadata',
'imageMe

In [135]:
l = ['kind',
'id',
'name',
'mimeType',
'description',
'starred',
'trashed',
'explicitlyTrashed',
'parents',
'properties',
'properties',
'appProperties',
'appProperties',
'spaces',
'version',
'webContentLink',
'webViewLink',
'iconLink',
'thumbnailLink',
'viewedByMe',
'viewedByMeTime',
'createdTime',
'modifiedTime',
'modifiedByMeTime',
'sharedWithMeTime',
'sharingUser',
'sharingUser',
'sharingUser',
'sharingUser',
'sharingUser',
'sharingUser',
'sharingUser',
'owners',
'owners',
'owners',
'owners',
'owners',
'owners',
'owners',
'lastModifyingUser',
'lastModifyingUser',
'lastModifyingUser',
'lastModifyingUser',
'lastModifyingUser',
'lastModifyingUser',
'lastModifyingUser',
'shared',
'ownedByMe',
'viewersCanCopyContent',
'writersCanShare',
'permissions',
'folderColorRgb',
'originalFilename',
'fullFileExtension',
'fileExtension',
'md5Checksum',
'size',
'quotaBytesUsed',
'headRevisionId',
'contentHints',
'contentHints',
'contentHints',
'contentHints',
'contentHints',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'imageMediaMetadata',
'videoMediaMetadata',
'videoMediaMetadata',
'videoMediaMetadata',
'videoMediaMetadata',
'capabilities',
'capabilities',
'capabilities',
'capabilities',
'capabilities',
'capabilities',
'isAppAuthorized',
'hasThumbnail',
'thumbnailVersion',
'modifiedByMe',
'trashingUser',
'trashingUser',
'trashingUser',
'trashingUser',
'trashingUser',
'trashingUser',
'trashingUser',
'trashedTime',
'teamDriveId',
'capabilities',
'capabilities',
'capabilities',
'capabilities',
'capabilities',
'capabilities',
'capabilities',
'capabilities',
'capabilities',
'capabilities',
'capabilities',
'hasAugmentedPermissions',
'capabilities',
'permissionIds',
'capabilities',
'copyRequiresWriterPermission',
'capabilities',
'capabilities',
'capabilities',
'capabilities',
'capabilities',
'capabilities',
'exportLinks',
'exportLinks',
'driveId',
'capabilities',
'capabilities',
'capabilities',
'capabilities',
'capabilities',
'capabilities',
'capabilities',
'capabilities',
'shortcutDetails',
'shortcutDetails',
'shortcutDetails',
'capabilities',
'capabilities',
'contentRestrictions',
'contentRestrictions',
'contentRestrictions',
'contentRestrictions',
'contentRestrictions',
'contentRestrictions',
'contentRestrictions',
'contentRestrictions',
'contentRestrictions',
'contentRestrictions',
'contentRestrictions',
'contentRestrictions',
'capabilities',
'shortcutDetails',
'resourceKey',
'linkShareMetadata',
'linkShareMetadata',
'linkShareMetadata',
'capabilities',
'Methods',
'copyCreates',]

In [137]:
s = set(l)

In [138]:
s

{'Methods',
 'appProperties',
 'capabilities',
 'contentHints',
 'contentRestrictions',
 'copyCreates',
 'copyRequiresWriterPermission',
 'createdTime',
 'description',
 'driveId',
 'explicitlyTrashed',
 'exportLinks',
 'fileExtension',
 'folderColorRgb',
 'fullFileExtension',
 'hasAugmentedPermissions',
 'hasThumbnail',
 'headRevisionId',
 'iconLink',
 'id',
 'imageMediaMetadata',
 'isAppAuthorized',
 'kind',
 'lastModifyingUser',
 'linkShareMetadata',
 'md5Checksum',
 'mimeType',
 'modifiedByMe',
 'modifiedByMeTime',
 'modifiedTime',
 'name',
 'originalFilename',
 'ownedByMe',
 'owners',
 'parents',
 'permissionIds',
 'permissions',
 'properties',
 'quotaBytesUsed',
 'resourceKey',
 'shared',
 'sharedWithMeTime',
 'sharingUser',
 'shortcutDetails',
 'size',
 'spaces',
 'starred',
 'teamDriveId',
 'thumbnailLink',
 'thumbnailVersion',
 'trashed',
 'trashedTime',
 'trashingUser',
 'version',
 'videoMediaMetadata',
 'viewedByMe',
 'viewedByMeTime',
 'viewersCanCopyContent',
 'webContent